# Very Large Numbers

Sometimes you want to work with very large numbers, for instance when you are doing Statistical Mechanics, or when looking for very large primes. These numbers are too large to be represented by a normal floating point numbers, leading to problems with your calculation. This notebook does a brief exploration what you can and cannot do in Python.

The details of how numbers are represented on a computer is a rather complicated bit of knowledge, which we usually try to avoid worrying about. But it is good to know some minimal information.

## Python Numeric Types

You can read more about the different Python numeric types in the manual pages: [Numeric Types](https://docs.python.org/3/library/stdtypes.html#numeric-types-int-float-complex)

The basic types in Python are `int`, `float` and `complex`. 

## Floating Point
Let's take a closer look at `float`, one of the main numerical types in just about any programming language. 
(_Note:_ In C/C++/Java this is called a `double`, which is usually 64-bit, while a `float` is 32-bit, but those are details.)

In [1]:
import math
import sys
print(sys.float_info)  # The sys package allow you to querry items about 
the_biggest_float = sys.float_info.max
print(the_biggest_float,the_biggest_float*1.01,the_biggest_float+10e+291)
print(math.pow(10,sys.float_info.max_10_exp))
print(math.pow(2,1023))
math.pow(10,sys.float_info.max_10_exp+1)

sys.float_info(max=1.7976931348623157e+308, max_exp=1024, max_10_exp=308, min=2.2250738585072014e-308, min_exp=-1021, min_10_exp=-307, dig=15, mant_dig=53, epsilon=2.220446049250313e-16, radix=2, rounds=1)
1.7976931348623157e+308 inf inf
1e+308
8.98846567431158e+307


OverflowError: math range error

So what this all tells us is that the largest number that can be represented by a `float` in Python is just a little larger than $10^{308}$. This is a pretty big number, so usually this is not a problem. Note that this limit is not only true for Python but for _any_ program that uses your current CPU, which goes up to 64-bit floating point numbers, _unless_ you do something fancy with your programming.

### Very Large Floating Point

If you need very large floating point numbers, or need very _precise_ floating point numbers, you can use the Python package [decimal](https://docs.python.org/3/library/decimal.html). This package allows _you_ to specify the maximum exponent and the precision of the numbers, but doing so increases the CPU time and memory required for each step in a calculation. Below an example of calculating $e^{1234567890}$, with 500 digits in precision.


In [29]:
from decimal import *
getcontext().prec=500       # Set the desired precision: keep 500 digits.
getcontext().Emax=1000000000 # Set the desired maximum exponent to 10^9 (1 billion).
Decimal.exp(Decimal(1234567890))

Decimal('1.4519136659370209179014008704498698351281948153941695118189330797423959619169464426506424578876161202481707340780996073728712898632165153768658854155376874993000385227087012559374363129384660851802921771117880623862644866892230338123725829854686245379329399766520351294415649903496862206655817322101573577273661067671260058389722073951653801432262766955933798951244730561195735555501086035624113320700695067373009119961865587986394805562452773612840601291505054721689833591924956429108664837898644800E+536166022')

Is there a still a limit?

The answer to that question is: it depends. If you are using the accelerated C version of the module, which comes with Python 3, then the maximum exponent is MAX_EMAX=999999999999999999, and the maximum precision is MAX_PREC=999999999999999999. Pretty large, but not infinite. In the "Pure Python" version there is supposedly no maximum, other than your patience waiting for the result...

There are other limitation to the `decimal` package, the largest one is that many math functions (cos, sin, etc) are not implemented to use the very high precision available. 

## mpmath 

An alternative to `decimal` (which is part of standard Python) is the package [mpmath](http://mpmath.org), which is included in the Anaconda distribution. This package allows you to do arbitrary-precision floating-point arithmetic, and it _will_ allow to go and spend large amounts of CPU time to compute those elusive first million digits of $\pi$: (My computer, 18 seconds)


In [11]:
import mpmath as m
m.mp.dps = 100  # This means 100 digits of precision
print(m.pi)     # Print pi to this precision.
f = 3/7
print(f)
f = m.mpf(3)/7  # Here we define 3 to be a 100 digit precise version, and now the result will be equally precise.
print(f)

3.141592653589793238462643383279502884197169399375105820974944592307816406286208998628034825342117068
0.42857142857142855
0.4285714285714285714285714285714285714285714285714285714285714285714285714285714285714285714285714286


When you want to combine mpmath with numpy, you will often need to be a bit careful with the order that you create numpy arrays. Unfortunately, you cannot go: `mpf( np.linspace(0,1,100))` to get 0 to 1 in 100 steps with high accuracy, because `mpf()` does not accept an array. If you do `[ mpf(x) for x in np.linspace(0,1,100)]`, you get a Python array of accurate numbers, but now it is no longer a numpy array. So to get the correct numpy array you need to do: `x = np.array( [m.mpf(x) for x in np.linspace(0,1,N)])`. This is a bit cumbersome, but very accurate.

You can now do algebra with `x`, such as `(1/x)**10000` without a problem, but unfortunately, you stil cannot do: `np.sin(x)`, or `m.sin(x)`. Both fail. Check your understanding why! So to get the sin for the entire array, and put that back in to a numpy array, you need to do this the slow way: `s=np.array([ m.sin(q) for q in x )`. 

See the example below:

In [31]:
import mpmath as m
import numpy as np
m.mp.dps = 100
N=100
x = np.array( [m.mpf(x) for x in np.linspace(0,1,N)])
print(x[N//2])
print( ((4*x*(x-1))**10000)[N//2] )   # Now you can do numpy style operations with the result
# print( np.sin(x[N//2]))             # This gives an error!
print( m.sin(x[N//2]))                # This works, but it is only the sin() of one element in the numpy array.
s = np.array([ m.sin(q) for q in x])  # Slow but accurate computation of  sin() for each x.
print( s[N//2])

0.50505050505050508302673506477731280028820037841796875
0.3604665533131356728009602392137915889961014557717633684080757043234434859572246338328539477040136851
0.4838516404379346554628391570642960369097171750636959655681900562328036156004242287914765294417569535
0.4838516404379346554628391570642960369097171750636959655681900562328036156004242287914765294417569535



### Exercises

   - Try computing $e^{e^{e^e}}$. 
   - How many times does the sequence '12345' appear in the first million digits of $\pi$?
   - Plot $4 x(x-1)^{N}$ for $0<x<1$ and $N=1,10,100,1000,10000$.

## Integers

How about integers? It turns out that for Python, the integer is a special class, which behaves a bit differently from the `int` in C/C++/Java. It is implemented to simply never stop cranking out more digits. It can take a while to compute all those digits, and it can take a whole lot longer to print them, so be careful with really big computations. If you get tired of waiting, you can interrupt your code by pressing the square stop button in the control bar at the top of your Jupyter notebook, or typing crtl-c in a terminal session.

#### Example: 
Count the number of digits in $2^{2^{2^{2^2}}}$

In [3]:
len(str(2**(2**(2**(2**2)))))

19729

If you square that number _again_, expect to wait a while.

## Factorials

Since Python can handle very large integers, it can handle very large factorials (see [math](https://docs.python.org/3.7/library/math.html)). Again, here you need to be careful what you print to the screen, since 100000! already has 456574 digits (try it.) You also need to be careful about how much CPU you are using to compute such big numbers. Add one more zero (factorial of a million) and your wait time for the computation goes to ~10 s for the factorial, but turning that large number into a string takes much longer (~7 mins). You end up with over 5 million digits.

### Binomial Coefficient

In Statistical Mechanics you will often need to compute the [binomial coefficient](https://en.wikipedia.org/wiki/Binomial_coefficient):

$$
\left(\begin{array}{c} n \\ k \end{array}\right) = \frac{n!}{k! (n-k)!}
$$
which can require large factorial numbers. Let's look at the different ways we can calculate this in Python.

#### Case 1 - the _result_ is less than 1e+308

If the _result_ of the calculation fits within the largest floating point number available, then you can calculate the binomial coefficient using scipy.special.binom(n,k). This is possible even if $n!$ is too large to fit in a float number. You can also compute the binomial coeffient your self using math.factorial, but this is often a lot slower for large n.

Here is a bit of code that compares the two:

In [4]:
from math import factorial as fac
from scipy.special import binom

def my_binom(n,k):
    return( fac(n)//(fac(k)*fac(n-k)))  # Note the use of // to make sure we get integer division in Python 3.

n = 1000
k = 200
exact = my_binom(n,k)
res1 = binom(n,k)
print(res1,float(exact)," Fractional error = ",(res1-exact)/exact)

6.6171555606649396e+215 6.6171555606593036e+215  Fractional error =  8.51727318051182e-13


The scipy function is not _quite_ as accurate as the integer version, but a whole lot faster for large $n$.

#### Case 2 - the _result_ is larger than 1e+308

In cases where your result is larger than 1e+308, you are dealing with rather unwieldy numbers. We can now use `mpmath` to still get an answer in exponential form, or we can use our pure integer method from before, or we can use the _symbolic python_ package `sympy`. The advantage of using `sympy` is that it evaluates a whole lot faster.

Below are examples:

In [5]:
import mpmath
  # This sets the number of digit accuracy that is kept. If you set dps high enough, anser is exact.
mpmath.mp.dps=20       
n=3000
k=200
exact = my_binom(n,k)
res2=mpmath.binomial(n,k)
print(res2," Fractional error = ",(res2-exact)/exact)

3.806029324345853125e+317  Fractional error =  6.9956863078732765278e-23


In [7]:
import sympy
N,K = sympy.symbols("N,K")
def sym_binom(n,k):
    b = sympy.factorial(N)/(sympy.factorial(K)*sympy.factorial(N-K))
    return(b.subs([(N,n),(K,k)]))
n=300000
k=2000
print("time mpmath.binomial(n,k)")
%time res2=mpmath.binomial(n,k)
print("time sym_binom(n,k)")
%time res3=sym_binom(n,k)
print("time my_binom(n,k)")
%time res4=my_binom(n,k)
print(res2)
print("mpmath fractional error = ",(res2-res4)/res4)
print("sympy  fractional error = ",(res3-res4)/res4)

time mpmath.binomial(n,k)
CPU times: user 317 µs, sys: 0 ns, total: 317 µs
Wall time: 322 µs
time sym_binom(n,k)
CPU times: user 2.18 s, sys: 6.83 ms, total: 2.18 s
Wall time: 2.18 s
time my_binom(n,k)
CPU times: user 2.29 s, sys: 10.7 ms, total: 2.3 s
Wall time: 2.3 s
6.6306990001136757305e+5215
mpmath fractional error =  3.7101806642031799036e-22
sympy  fractional error =  0


#### Case 3 - Practical considerations

These results are numbers that too large to do anything useful with them. If you wanted to make a plot, you would probably choose a log-y scale, in which case it is probably more useful to calculate the _log_ values of these numbers. In that case, you can use the approximation:

$$
N! \approx N \log(N) - N
$$

We can use Python to check how good this approximation is for numbers $10^x$:

In [9]:
import mpmath
import sympy

mpmath.mp.dps=26
def compare1(n):
    exact = mpmath.log(mpmath.factorial(N))
    approx = N*mpmath.log(N) - N
    return( (exact-approx)/exact )

N = sympy.symbols("N")
def compare2(n):
    exact = sympy.log(sympy.factorial(N))
    approx = N*sympy.log(N) - N
    R = (exact - approx)/exact 
    return( sympy.N(R.subs(N,n)))

for x in range(1,26):
    N=mpmath.power(10,x)
    dif1 = compare1(N)
    dif2 = compare2(N)
    print("10^{:<3d} err1 = {:10.4e}  err2 = {:10.4e} ".format(x,float(dif1),dif2))

10^1   err1 = 1.3761e-01  err2 =  1.3761e-1 
10^2   err1 = 8.8590e-03  err2 =  8.8590e-3 
10^3   err1 = 7.3965e-04  err2 =  7.3965e-4 
10^4   err1 = 6.7278e-05  err2 =  6.7278e-5 
10^5   err1 = 6.3497e-06  err2 =  6.3497e-6 
10^6   err1 = 6.1072e-07  err2 =  6.1072e-7 
10^7   err1 = 5.9386e-08  err2 =  5.9386e-8 
10^8   err1 = 5.8145e-09  err2 =  5.8145e-9 
10^9   err1 = 5.7194e-10  err2 = 5.7194e-10 
10^10  err1 = 5.6442e-11  err2 = 5.6442e-11 
10^11  err1 = 5.5832e-12  err2 = 5.5832e-12 
10^12  err1 = 5.5328e-13  err2 = 5.5328e-13 
10^13  err1 = 5.4904e-14  err2 = 5.4904e-14 
10^14  err1 = 5.4543e-15  err2 = 5.4543e-15 
10^15  err1 = 5.4231e-16  err2 = 5.4231e-16 
10^16  err1 = 5.3959e-17  err2 = 5.3959e-17 
10^17  err1 = 5.3720e-18  err2 = 5.3720e-18 
10^18  err1 = 5.3508e-19  err2 = 5.3508e-19 
10^19  err1 = 5.3319e-20  err2 = 5.3319e-20 
10^20  err1 = 5.3150e-21  err2 = 5.3150e-21 
10^21  err1 = 5.2996e-22  err2 = 5.2996e-22 
10^22  err1 = 5.2857e-23  err2 = 5.2857e-23 
10^23  err

So clearly, once you get to factorials of numbers larger than 100, the error in your calculation becomes quite small.
The log of the binomial function is then:

$$
\begin{array}{lcl}
\log\left(\begin{array}{c} n \\ k \end{array}\right) &=& \log\left(\frac{n!}{k!(n-k)!} \right)= \log(n!)-\log(k!) -\log((n-k)!)  \\
&\approx& n\log(n)-n - k\log(k) + k - (n-k)\log(n-k) + n - k  \\
&=& n\log(n) - k\log(k) - (n-k)\log(n-k)
\end{array}
$$

#### Exercise
Write a function that implements $\log\left(\begin{array}{c} n\\k\end{array}\right)$ using the approximation derived above, and convince yourself that this approximation is good by numerical evaluation.

## Big Prime Numbers

Another application of large integers is that of [prime numbers](https://en.wikipedia.org/wiki/Prime_number). According to the Wikipedia article: "A simple but slow method of checking the primality of a given number $n$, called trial division, tests whether $n$ is a multiple of any integer between 2 and $\sqrt {n}$. Faster algorithms include the Miller–Rabin primality test...". Another faster algorithm is the ["Sieve of Eratosthenes"](https://en.wikipedia.org/wiki/Sieve_of_Eratosthenes)(Ἐρατοσθένους), yes, dating back to the ancient Greeks.

Is you search the internet, you will find lots and lots of published snippets of code to calcuate prime numbers. Please be aware that many are not optimal, and a few are even incorrect. Below is an implementation that I wrote of the Sieve of Eratosthenes, where I tried to pay particular attention to making the algorithm run fast in Python. As a result the algorithm is more difficult to understand, but you may want to take the time to investigate how it works.


In [10]:
def primeSieve_slow(sieveSize):
    ''' Returns a list of prime numbers calculated using
    the Sieve of Eratosthenes algorithm.'''
    sieve = [True] * sieveSize
    sieve[0] = False # zero and one are not prime numbers
    sieve[1] = False
    # create the sieve
    for i in range(2, int(math.sqrt(sieveSize)) + 1): # You only need to sieve to sqrt(N)
        if sieve[i] == True:                          # Only need to check numbers that still may be primes.
            pointer = i * 2                           # This would be prime times 2, so not prime.
            while pointer < sieveSize:
                sieve[pointer] = False                # We thus mark it False
                pointer += i                          # And move to then next multiple of prime.
# compile the list of primes
    primes = []
    for i in range(sieveSize):
        if sieve[i] == True:
            primes.append(i)
    return primes

def primeSieve(sieveSize):
    ''' Returns a list of prime numbers calculated using
    the Sieve of Eratosthenes algorithm. Not practical for very
    large sieveSize. This version take ~1.5s for primes up to 10^7''' 
    # The following line does the same as the 5 lines above. Note how it treats sieveSize odd and even.
    sieve = [False,False,True,True]+[False,True]*(sieveSize//2 -2) + [False]*(sieveSize%2!=0)
    # create the sieve
    for i in range(3, int(math.sqrt(sieveSize)) + 1,2): # You only need to sieve every other up to sqrt(N)
        if sieve[i] == True:                          # Only need to check numbers that still may be primes.
#            for i in range(i*i,sieveSize,i):    # Mark every i-th value false, starting at i*i 
#                sieve[i]=False
#  This trick uses "slicing" of the list to quickly set particular values to False
            sieve[i*i::i]=[False]*((len(sieve)-i*i+i-1)//i) # len(sieve[i*i::i])
# compile the list of primes and return
    return [ i for i in range(sieveSize) if sieve[i]==True]  # This list comprehension version is faster.


In [11]:
%time primes=primeSieve(10000000)
print("Largest prime found = ",primes[-1]) # Largest prime found.

CPU times: user 1.11 s, sys: 138 ms, total: 1.25 s
Wall time: 1.25 s
Largest prime found =  9999991


For larger prime numbers, the [Rabin-Miller primality test](https://en.wikipedia.org/wiki/Miller%E2%80%93Rabin_primality_test) algorithm is useful. It does _not prove_ that a number is prime, it merely tells you that it is highly likely to be prime. 

This algorithm is quite complicated and only for the dedicated prime number searcher. Below is a simple implementation of this algorithm, and a routine that helps you check if a number is a prime. For low prime numbers it uses the primeSieve(), for larger ones Rabin-Miller.


In [12]:
import random
def rabinMiller(num):
    '''
    Rabin Miller primality test - https://en.wikipedia.org/wiki/Miller%E2%80%93Rabin_primality_test
    
    Returns True if num is a probably a prime number. Since it uses trial
    functions, there is no guarantee. It is quite good though, and usually 
    the number is indeed prime if this returns true.'''
    s = num - 1
    t = 0
    while s % 2 == 0:
        # keep halving s while it is even (and use t
        # to count how many times we halve s)
        s = s // 2
        t += 1

    for trials in range(15): # try to falsify num's primality 15 times
        a = random.randrange(2, num - 1)
        v = pow(a, s, num)
        if v != 1: # this test does not apply if v is 1.
            i = 0
            while v != (num - 1):
                if i == t - 1:
                    return False
                else:
                    i = i + 1
                    v = (v ** 2) % num
    return True

def isProbablyPrime(num):
    global Low_Primes
    # Return True if num is a prime number. This function does a quicker
    # prime number check before calling rabinMiller().

    if (num < 2):
        return False # 0, 1, and negative numbers are not prime

    # About 1/3 of the time we can quickly determine if num is not prime
    # by dividing by the first few dozen prime numbers. This is quicker
    # than rabinMiller(), but unlike rabinMiller() is not guaranteed to
    # prove that a number is prime.
    if not 'Low_Primes' in globals():
        Low_Primes = primeSieve(1000000)

    if num in Low_Primes:
        return True

    # See if any of the low prime numbers can divide num
    for prime in Low_Primes:
        if (num % prime == 0):
            return False

    # If all else fails, call rabinMiller() to determine if num is a prime.
    return rabinMiller(num)

def generateLargePrime(keysize=1024):
    '''Return a random prime number that has keysize bits in size.''' 
    while True:
        num = random.randrange(2**(keysize-1), 2**(keysize))
        if isProbablyPrime(num):
            return num
        

We can now test _very_ large integers on whether they are prime numbers. The people that search for extremely large primes usually search for [Mersenne primes](https://en.wikipedia.org/wiki/Mersenne_prime), which are all of the form: $2^{N}-1$. The [largest known prime](https://en.wikipedia.org/wiki/Largest_known_prime_number), found in December 2017, is:

$$
2^{77232917}-1
$$

I leave it up to you to count how many digits it has. (Hint: don't try to print this!)

The algorithm below finds the first Mersenne primes up to $2^{200}-1$.

In [13]:
for i in range(1,200):
    num = 2**i-1
    if isProbablyPrime(num):
        num_digits = int(mpmath.log10(num))+1 
        print("2^{:<4d}-1 is a prime with {:4d} digits.".format(i,num_digits))

2^2   -1 is a prime with    1 digits.
2^3   -1 is a prime with    1 digits.
2^5   -1 is a prime with    2 digits.
2^7   -1 is a prime with    3 digits.
2^13  -1 is a prime with    4 digits.
2^17  -1 is a prime with    6 digits.
2^19  -1 is a prime with    6 digits.
2^31  -1 is a prime with   10 digits.
2^61  -1 is a prime with   19 digits.
2^89  -1 is a prime with   27 digits.
2^107 -1 is a prime with   33 digits.
2^127 -1 is a prime with   39 digits.


### Exersizes:
  - Calculate the number of digits for the largest known prime number. If you are smart about it, this takes less than a second. Check your answer against the Wikipedia article.
  - Mersenne primes cannot be the only set of big primes "out there". Look at how successful an alternate set of primes is: $2^N - 3$. For $N<=400$, are there more Mersenne primes, or more alternate primes? (Perhaps we should _name_ them?)
  - Another interesting topic in large integers is that of ["perfect numbers"](https://en.wikipedia.org/wiki/Perfect_number). There are far fewer perfect numbers than primes. A currently still unsolved problem in mathematics is whether there is a finite or infinite set of perfect numbers. Show by direct computation that the first 12 perfect numbers are of the form:
  $$ 2^{p-1}\left(2^p-1\right) $$
  where $\left(2^p-1\right)$ is a prime number.

## Answers to exercises:


In [15]:
# Try e^e^e^e  using decimal
import decimal as d
from decimal import Decimal as D   # Save some typing of d.Decimal()
d.getcontext().Emax = d.MAX_EMAX   # Allow for the very largest decimals
D.exp(D.exp(D.exp(D.exp(D(1)))))   # Note that the argument must be a Decimal

Decimal('2.3315043990071954622896899110121376663320174289635168232800545468180794366424973148573066613214076731450395088188518894592235976706744943602673067889669028720251433852975259374232282055849564334055538196517479723893702683644040304673109438136348628130600224362014430607799345804846171068807958573994931475104930480457442412215968068457840685457248750994424008425171988575149328910345012794771119508157927743364552242751642079275602685116082577801807502074737791924997582364652909840292013082102887271E+1656520')

In [17]:
# Try e^e^e^e  using mpmath
import mpmath as M
M.mp.dps=300                      # Give 300 digits of precision
M.exp(M.exp(M.exp(M.exp(1))))

mpf('2.33150439900719546228968991101213766633201742896351682328005454681807943664249731485730666132140767314503950881885188945922359767067449436026730678896690287202514338529752593742322820558495643340555381965174797238937026836440403046731094381363486281306002243620144306077993458048461710688079585124324769e+1656520')

In [18]:
# How many times does the sequence '12345' appear in the first million digits of $\pi$?
#
# Note, you really, really, do not want to print all that to your screen!!
#
import mpmath as M
M.mp.dps=1000000                 # One million digits of precision (ouch)
%time bigpi = str(M.acos(0)*2)   # Calculate pi as the inverse cosine. 
len(bigpi)                       # Do I have 1 million digits, really?
print("# of times 12345 in pi =",bigpi.count('12345'))             # Count the number of time 12345 occurs.
#
# Prepare to wait a while
#

CPU times: user 13min 10s, sys: 1.54 s, total: 13min 11s
Wall time: 13min 12s
# of times 12345 in pi = 8


In [19]:
#
# Evaluate log( binom(n,k)) directly and compare with the approximation n log(n) - k log(k) - (n-k)log(n-k)
#
from math import log
import mpmath as M
M.mp.dps=100  
for x in range(4,10):
    for y in range(4,x):
        n = M.power(10,x)
        k = M.power(10,y)
        exact = M.log(M.binomial(n,k))
        approx= n*log(n) - k*log(k) - (n-k)*log(n-k)
        err = (approx-exact)/exact
        print("Binomial(10^{:<2d},10^{:2d}) Fracional error = {:10.4e}".format(x,y,float(err)))

Binomial(10^5 ,10^ 4) Fracional error = 1.6834e-04
Binomial(10^6 ,10^ 4) Fracional error = 9.8562e-05
Binomial(10^6 ,10^ 5) Fracional error = 2.0373e-05
Binomial(10^7 ,10^ 4) Fracional error = 6.9860e-05
Binomial(10^7 ,10^ 5) Fracional error = 1.1911e-05
Binomial(10^7 ,10^ 6) Fracional error = 2.3914e-06
Binomial(10^8 ,10^ 4) Fracional error = 5.4106e-05
Binomial(10^8 ,10^ 5) Fracional error = 8.4416e-06
Binomial(10^8 ,10^ 6) Fracional error = 1.3967e-06
Binomial(10^8 ,10^ 7) Fracional error = 2.7455e-07
Binomial(10^9 ,10^ 4) Fracional error = 4.4149e-05
Binomial(10^9 ,10^ 5) Fracional error = 6.5379e-06
Binomial(10^9 ,10^ 6) Fracional error = 9.8975e-07
Binomial(10^9 ,10^ 7) Fracional error = 1.6023e-07
Binomial(10^9 ,10^ 8) Fracional error = 3.0997e-08


In [20]:
#
# Number of digits for the largest known prime:
#
import mpmath as M
%time int(mpmath.log10(2**77232917-1))+1

CPU times: user 300 ms, sys: 18.7 ms, total: 319 ms
Wall time: 317 ms


23249425

In [21]:
# Count Mersenne and alternate primes:
mp_list=[]
mp_alt_list=[]
for i in range(1,400):
    mp = 2**i-1
    mp_alt = 2**i-3
    if isProbablyPrime(mp):
        mp_list.append(i)
    if isProbablyPrime(mp_alt):
        mp_alt_list.append(i)
print("There are {} Mersenne primes, and {} alt primes for N<=400".format(len(mp_list),len(mp_alt_list)))

There are 12 Mersenne primes, and 22 alt primes for N<=400
